In [ ]:
import numpy as np

In [ ]:
# 定义变量
m = 1000 
theta = 0.75
alp = 1.0

In [ ]:
# 定义文件夹路径
folder = 'input/'
folder1 = 'result/'

In [ ]:
# 打开拓扑文件
with open(folder + 'RIVER_TOPO_FILE.txt') as f:
    JuncNum, JuncNum1, JuncNum2 = [int(x) for x in f.readline().split()] 
    ChanNum, chanNum1, chanNum2 = [int(x) for x in f.readline().split()]
    UbouNum, AbouNum = [int(x) for x in f.readline().split()]